In [1]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import os

In [2]:
submissions = []
for submission in os.listdir('../submissions/'):
    model_type = submission.split('_')[0]
    if model_type not in ("rank", "ridge", "logisticstack"):
        submissions.append(submission)

# OOF Data

In [3]:
n_val = len(pd.read_csv("../input/train.csv", usecols=["target"]))
val_data = np.zeros((n_val, len(submissions)))
column_names = []
for i, submission in enumerate(submissions):
    column_names.append(submission.split('.csv')[0])
    val_data[:,i] = pd.read_csv(r"../output/oof predictions/{}".format(submission), 
                            usecols=["target"]).values[:,0]
oof_labels = pd.read_csv("../input/train.csv", usecols=["target"]).values[:,0]
clf = LogisticRegression(random_state=42)
clf.fit(val_data, oof_labels)
val_preds = clf.predict_proba(val_data)[:,1]
val_auc  = roc_auc_score(oof_labels, val_preds)
print('Validation AUC before stretch: {}'.format(val_auc))
val_preds = (val_preds - val_preds.min()) / (val_preds.max() - val_preds.min())
val_auc  = roc_auc_score(oof_labels, val_preds)
print('Validation AUC after stretch: {}'.format(val_auc))

/home/apg/all_dl/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Validation AUC before stretch: 0.9017470042642681
Validation AUC after stretch: 0.9017470042642681


In [4]:
clf.coef_

array([[ 0.16496851,  2.50557337, -0.58037078,  1.9081381 ,  0.0497083 ,
         0.01197462,  0.20523732,  0.39422658,  0.19057639,  0.35266549,
         0.61240815, -0.48560536, -0.24919708,  1.94169559]])

# Test Data

In [5]:
n_test = len(pd.read_csv("../input/test.csv", usecols=["ID_code"]))
test_data = np.zeros((n_test, len(submissions)))
column_names = []
for i, submission in enumerate(submissions):
    column_names.append(submission.split('.csv')[0])
    test_data[:,i] = pd.read_csv("../submissions/{}".format(submission), 
                               usecols=["target"]).values[:,0]
test_preds = clf.predict_proba(test_data)[:,1]

In [6]:
test_preds.min(), test_preds.max()

(0.007974906977581167, 0.9114834801610247)

In [7]:
test_preds = (test_preds - test_preds.min()) / (test_preds.max() - test_preds.min())

In [8]:
test_preds.min(), test_preds.max()

(0.0, 1.0)

In [9]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['target'] = test_preds
submission.to_csv('../submissions/stack.csv', index=False)